In [1]:
import os,sys
from pathlib import Path
currentdir = Path(os.getcwd())
sys.path.insert(0, os.path.dirname(currentdir.parent))
sys.path.insert(0, os.path.dirname(currentdir))
print(str(sys.path[0:3]))

import pandas as pd
from varclushi import VarClusHi

data = pd.read_csv('data/arth150.csv')
vc = VarClusHi(data, maxeigval2=1, maxclus=4)
vc.varclus()
vc.info

['/home/jerzy/Documents/GitHub/GitHub/FEDOT', '/home/jerzy/Documents/GitHub/GitHub', '/home/jerzy/Documents/GitHub/GitHub/FEDOT/examples']


KeyboardInterrupt: 

In [9]:
vc.rsquare

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,13,0.536863,0.052427,0.488761
1,0,38,0.017625,0.000298,0.982668
2,0,47,0.673910,0.226817,0.421750
3,0,61,0.006970,0.000149,0.993178
4,0,81,0.006267,0.000163,0.993895
...,...,...,...,...,...
102,3,444,0.693018,0.000208,0.307046
103,3,540,0.603780,0.000039,0.396235
104,3,636,0.261741,0.000373,0.738534
105,3,738,0.854156,0.000065,0.145854


In [14]:
clusters_df = vc.rsquare
clusters_df = clusters_df[['Cluster', 'Variable']]
clusters_df

,Cluster,Variable
0,0,13
1,0,38
2,0,47
3,0,61
4,0,81
...,...,...
102,3,444
103,3,540
104,3,636
105,3,738
